# Environmental and Societal Factors

> A collection of visualizations to highlight how environmental and societal factors impact our mental well-being.

- toc: true
- badges: false
- comments: false

In [34]:
# hide
# Code Import

## Imports:

### Libraries
import numpy as np
import pandas as pd
import altair as alt
import os, os.path
import pycountry
from vega_datasets import data

alt.data_transformers.enable('csv')

import country_converter as coco
import geopandas as gpd
import json

### Data
cwd = os.getcwd()
df_file = '/depression_final_df.csv'
df = pd.read_csv(cwd + df_file)

countries = alt.topo_feature(data.world_110m.url, 'countries')

df = df.dropna(subset=['id'])
df.id = df.id.astype(int)
df = df.rename(columns={"Prevalence - Depressive disorders - Sex: Male - Age: Age-standardized (Percent)": "Depression in males (%)",
                        "Prevalence - Depressive disorders - Sex: Female - Age: Age-standardized (Percent)": "Depression in females (%)"})

sum_column = df["Depression in males (%)"] + df["Depression in females (%)"]

df['Depression (%)'] = sum_column

In [70]:
df2

,Country,Year,Depression in males (%),Depression in females (%),Population (historical estimates),Access to green space,Employment rate,Life satisfaction,Perceived health,Satisfaction with time use,Social support,Time spent in social interactions,Divorce Rate,Depression (%)
0,Fiji,1990,2.565040,3.627058,728575.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.192097
1,Fiji,1991,2.574759,3.637601,735398.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.212360
2,Fiji,1992,2.583960,3.647769,744470.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.231729
3,Fiji,1993,2.594349,3.657140,754962.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.251489
4,Fiji,1994,2.602736,3.663786,765607.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.266522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4734,Trinidad and Tobago,2013,2.869620,3.808907,1353708.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.678528
4735,Trinidad and Tobago,2014,2.870082,3.805982,1362337.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.676064
4736,Trinidad and Tobago,2015,2.867165,3.800037,1370332.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.667202
4737,Trinidad and Tobago,2016,2.864625,3.794836,1377563.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.659460


In [78]:
# hide
df_1990 = df[df['Year'] == 1990]
df_2000 = df[df['Year'] == 2000]
df_2010 = df[df['Year'] == 2010]
df_2017 = df[df['Year'] == 2017]

df2 = df.drop(columns=['country_code','geometry','Continent','id']) 
df2 = df2.fillna(0)
df2['Year'] = df['Year'].apply(int)

df3 = pd.melt(df2, id_vars=['Country','Year'], var_name=['Statistic'])
df3

,Country,Year,Statistic,value
0,Fiji,1990,Depression in males (%),2.565040
1,Fiji,1991,Depression in males (%),2.574759
2,Fiji,1992,Depression in males (%),2.583960
3,Fiji,1993,Depression in males (%),2.594349
4,Fiji,1994,Depression in males (%),2.602736
...,...,...,...,...
56131,Trinidad and Tobago,2013,Depression (%),6.678528
56132,Trinidad and Tobago,2014,Depression (%),6.676064
56133,Trinidad and Tobago,2015,Depression (%),6.667202
56134,Trinidad and Tobago,2016,Depression (%),6.659460


In [72]:
#hide
click = alt.selection_multi(fields=['Country'], empty='none')

main_width = 675
main_height = 350
main_colour = alt.Color('Depression (%):Q',scale=alt.Scale(scheme='viridis',reverse=True))

main_choropleth = (alt.Chart(countries).mark_geoshape().transform_lookup(
    lookup='id',
    from_=alt.LookupData(df_2017, 'id', ['Depression (%)','Country']))
.encode(
    color=main_colour, #'Schizophrenia (%):Q',
    opacity=alt.condition(click, alt.value(1), alt.value(0.2)),
    tooltip=['Country:N', 'Depression (%):Q'])
.add_selection(click)
.project(
    type='equirectangular')
).properties(
    width=main_width,
    height=main_height
)

main_background = alt.Chart(countries).mark_geoshape(
    fill='lightgrey',
    stroke='black',
    strokeWidth=0.1
).project(
    'equirectangular'
).properties(
    width=main_width,
    height=main_height
)

main_map = (main_background + main_choropleth )

## Map Select

In [73]:
# hide_input
main_map

alt.LayerChart(...)

In [76]:
stats = alt.Chart(df3).mark_area().encode(
    x='Year:T',
    y='value:Q',
    color='Statistic:N',
    row=alt.Row('Statistic:N') #, sort=['MSFT', 'AAPL', 'IBM', 'AMZN'])
).properties(height=1000, width=600).transform_filter(
    click
)

In [77]:
main_map & stats

alt.VConcatChart(...)